In [16]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_2'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

#print(Y_all)
#print(X_all)
Y = Y_all['2002-06-04':' 2016-12-31']
X = X_all['2002-05-05':' 2016-12-31']
# 2017-01-01 to 2021-12-31 testing # we don't have target variables for the "testing phase"
# for sweden weekly forecasts are asked
print(X.index, Y.index)

DatetimeIndex(['2002-05-05', '2002-05-06', '2002-05-07', '2002-05-08',
               '2002-05-09', '2002-05-10', '2002-05-11', '2002-05-12',
               '2002-05-13', '2002-05-14',
               ...
               '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
               '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
               '2016-12-30', '2016-12-31'],
              dtype='datetime64[ns]', name='time', length=5355, freq=None) DatetimeIndex(['2002-06-04', '2002-06-11', '2002-06-18', '2002-06-25',
               '2002-07-02', '2002-07-09', '2002-07-16', '2002-07-23',
               '2002-07-30', '2002-08-06',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', name='Date', length=709, freq=None)


In [17]:
X_train = X[:'2011-12-26']
Y_train = Y[:'2011-12-27']
print(X_train.index, Y_train.index)

X_test = X['2014-01-05':'2014-12-29']
Y_test = Y['2014-02-04':'2014-12-30']#29
print(X_test.index, Y_test.index)

X_valid = X['2012-01-01':'2013-12-30']
Y_valid = Y['2012-01-31':'2013-12-31']
#print(X_valid.index, Y_valid.index)
print(Y_train.shape, Y_test.shape, Y_valid.shape)

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

#Preprocessing nur Warning, kein Fehler, normierung wird korrekt durchgeführt
(x_tr,y, x_te, y_test, x_va, y_valid, scaler_X,scaler_Y)=prepare_data_weekly(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)
print(x_te.shape)
print(x_tr.shape)
print(x_va.shape)

iters=3
n_steps_in=30
n_steps_out=1

x = split_sequences_x(x_tr, n_steps_in, n_steps_out)
x_test = split_sequences_x(x_te, n_steps_in, n_steps_out)
x_valid = split_sequences_x(x_va, n_steps_in, n_steps_out)
print(x_test.shape, x.shape, x_valid.shape)
print(y_test.shape, y.shape, y_valid.shape)

DatetimeIndex(['2002-05-05', '2002-05-06', '2002-05-07', '2002-05-08',
               '2002-05-09', '2002-05-10', '2002-05-11', '2002-05-12',
               '2002-05-13', '2002-05-14',
               ...
               '2011-12-17', '2011-12-18', '2011-12-19', '2011-12-20',
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-24',
               '2011-12-25', '2011-12-26'],
              dtype='datetime64[ns]', name='time', length=3523, freq=None) DatetimeIndex(['2002-06-04', '2002-06-11', '2002-06-18', '2002-06-25',
               '2002-07-02', '2002-07-09', '2002-07-16', '2002-07-23',
               '2002-07-30', '2002-08-06',
               ...
               '2011-10-25', '2011-11-01', '2011-11-08', '2011-11-15',
               '2011-11-22', '2011-11-29', '2011-12-06', '2011-12-13',
               '2011-12-20', '2011-12-27'],
              dtype='datetime64[ns]', name='Date', length=500, freq=None)
DatetimeIndex(['2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',


In [18]:
#transformer code is from keras example on time series classification
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x, training=True)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x, training=True)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    print('test1')
    pos_encoding = angle_rads[np.newaxis, ...]
    print('test2')
    return tf.cast(pos_encoding, dtype=tf.float32)

def build_model(
    input_shape,
    output_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)#change here enc/dec

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in [mlp_units]:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x, training=True)
    outputs = layers.Dense(output_shape, activation="relu")(x)
    return keras.Model(inputs, outputs)

In [19]:
from keras_tuner import HyperModel
from tensorflow.keras import layers

class Transformer_HyperModel(HyperModel):
    def __init__(self, input_shape, output_shape):
        super(Transformer_HyperModel, self).__init__()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
    def build(self, hp):#statt build_model
        head_size = hp.Int("head_size", min_value=8, max_value=64) #, default=16
        num_heads = 1
        ff_dim = hp.Int("ff_dim", min_value=1, max_value=8) #, default=4
        num_transformer_blocks = 2 #, default=4
        mlp_units = hp.Int("mlp_units", min_value=10, max_value=200)
        dropout = 0.1
        mlp_dropout = 0.1
        
        model = build_model(self.input_shape, self.output_shape, head_size,num_heads,ff_dim,
                                num_transformer_blocks,mlp_units,mlp_dropout)
        model.compile(loss="mean_absolute_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=["mean_absolute_error"])
        return model

input_shape = x.shape[1:]
output_shape = y.shape[1]

hyper_model = Transformer_HyperModel(input_shape, output_shape)

es_callback = keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error',restore_best_weights=True, patience=4)




In [20]:
tuner= kt.RandomSearch(
        hyper_model,
        overwrite=True,
        objective='val_mean_absolute_error',
        max_trials = 10,
        directory='D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge',
        project_name='TF Sweden2'
        )

tuner.search(
        x,
        y,
        batch_size=24,
        epochs=30,
        validation_data=(x_valid,y_valid),#achtung test-set hier darf nicht mein standard test set sein, sondern val-set!! 
        callbacks=[es_callback],
        verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]

Y_pred=best_model.predict(x_test)
best_model.summary()

Trial 10 Complete [00h 00m 05s]
val_mean_absolute_error: 0.1664285510778427

Best val_mean_absolute_error So Far: 0.13872769474983215
Total elapsed time: 00h 01m 34s
INFO:tensorflow:Oracle triggered exit
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        2427        layer_normalization[0][0]        
                                                                 layer_normalization[0][

In [21]:
tuner.results_summary()

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)
from tensorflow.keras.models import save_model
best_model.save('TF_Sweden2_tune.h5')

NSE=np.array([[None]*n_steps_out])
MAE=np.array([[None]*n_steps_out])
RMSE=np.array([[None]*n_steps_out])
SAPE=np.array([[None]*n_steps_out])

Y_pred_test=best_model.predict(x_test)
Y_pred_valid=best_model.predict(x_valid)

y_correct_utf = scaler_Y.inverse_transform(y_test)#test
yhat_utf = scaler_Y.inverse_transform(Y_pred_test)#test
Y_pred_valid_utf = scaler_Y.inverse_transform(Y_pred_valid)#valid

NSE[:]= np.array([1 - sum((yhat_utf[:,i]-y_correct_utf[:,i])**2)/sum((y_correct_utf[:,i]-np.mean(y_correct_utf[:,i]))**2)
                  for i in range(n_steps_out)])
MAE[:]=np.array([mean_absolute_error(y_correct_utf[:,i], yhat_utf[:,i]) for i in range(n_steps_out)])
RMSE[:]=np.array([np.sqrt(np.mean((yhat_utf[:,i]-y_correct_utf[:,i])**2)) for i in range(n_steps_out)])
SAPE[:] = np.array([np.abs((yhat_utf[:,i] - y_correct_utf[:,i])/(0.5*(y_correct_utf[:,i]+yhat_utf[:,i]))).mean()
                         for i in range(n_steps_out)]) #mean absolute percent error

print(np.mean(MAE))

Results summary
Results in D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\TF Sweden2
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
head_size: 62
ff_dim: 8
mlp_units: 156
Score: 0.13872769474983215
Trial summary
Hyperparameters:
head_size: 14
ff_dim: 2
mlp_units: 61
Score: 0.14910650253295898
Trial summary
Hyperparameters:
head_size: 35
ff_dim: 1
mlp_units: 68
Score: 0.16291871666908264
Trial summary
Hyperparameters:
head_size: 23
ff_dim: 4
mlp_units: 161
Score: 0.1664285510778427
Trial summary
Hyperparameters:
head_size: 23
ff_dim: 2
mlp_units: 60
Score: 0.16745036840438843
Trial summary
Hyperparameters:
head_size: 36
ff_dim: 2
mlp_units: 56
Score: 0.18067550659179688
Trial summary
Hyperparameters:
head_size: 28
ff_dim: 2
mlp_units: 37
Score: 0.18742665648460388
Trial summary
Hyperparameters:
head_size: 29
ff_dim: 5
mlp_units: 41
Score: 0.20826636254787445
Trial summary
Hyperparameters:
head_size: 9
ff_dim: 7
